In [ ]:
"""
[誤差逆伝播法(Back Propagation)]
勾配を数値微分で計算する方法は, シンプルで実装が容易だが, 計算に時間がかかるという難点がある.
逆伝播は計算グラフの各ノードにおいて, 局所的な微分を信号に乗算して, 次のノードに伝達していく.
この計算を行うことによって, 目的とする微分の値を効率よく求めることができる.

[連鎖律の原理]
ある関数が合成関数で表される場合, その合成関数の微分は, 合成関数を構成するそれぞれの関数の微分の積によって表すことができる.
これを連鎖律の原理という.

"""
pass

In [1]:
"""
乗算レイヤの実装
"""
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
    
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y
    
        return out
    
    def backward(self, dout):
        dx = dout * self.y # 順伝播のひっくり返した値を乗算して下流に流す
        dy = dout * self.x
        
        return dx, dy

In [5]:
"""
りんご２個の買い物
"""
apple = 100
apple_num = 2
orange_num = 3
tax = 1.1

# layer
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# forward
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print(price)

# backward
# 各変数に関する微分を求める

dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print("Diff for variables: ", dapple, "," , dapple_num, ",", dtax)

220.00000000000003
Diff for variables:  2.2 , 110.00000000000001 , 200


In [8]:
"""
加算レイヤの実装
"""
class AddLayer:
    
    def __init__(self):
        pass
    
    def forward(self, x, y):
        out = x + y
        return out
    
    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        return dx, dy

In [15]:
"""
りんご２個とみかん３個の買い物
"""
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# layer
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# forward
apple_price = mul_apple_layer.forward(apple, apple_num)
orange_price = mul_orange_layer.forward(orange, orange_num)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)
price = mul_tax_layer.forward(all_price, tax)

# backward
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print("Ouput Price:", price)
print("Diff of Apple num:", dapple_num)
print("Diff of Apple price:", dapple)
print("Diff of Orange num:", dorange_num)
print("Diff of Orange price:", dorange)
print("Diff of Tax:", dtax)

Ouput Price: 715.0000000000001
Diff of Apple num: 110.00000000000001
Diff of Apple price: 2.2
Diff of Orange num: 165.0
Diff of Orange price: 3.3000000000000003
Diff of Tax: 650
